## Importing the libraries

In [1]:
# import os library 
import os 
# Change the current directory 
# to specified directory 
os.chdir(r"C:Functions/") 

In [2]:
# Numpy
import numpy as np
# Import module for NURBS (geomdl)
from geomdl import BSpline, utilities
from geomdl.visualization import VisMPL as vis
# Matplot for plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
# Call the channel file and generate control points 
import orthogonalization as ort
# Call the NURBS file and generate the evaluation points of St
import NURBS
# Call the algortihm fuction
import algorithm as al
# Call the ECLIPSE writer to generate GRDECL.file 
import GRDECL_file as ecl

## Importing the file

In [ ]:
# Load txt file
a = np.loadtxt('../Channel_Data/garonne_1.txt', delimiter=',')
b = np.loadtxt('../Channel_Data/garonne_2.txt', delimiter=',')
c = np.loadtxt('../Channel_Data/garonne_3.txt', delimiter=',')

## Create the channel data points

In [ ]:
# Reshape only removing the id point
# Defining the array
data_1 = ort.dataset(a)
data_2 = ort.dataset(b)
data_3 = ort.dataset(c)

# Data point of channels
channel1 = ort.controlpoints(data_1.x,data_1.y,data_1.z,data_1.w,data_1.array)
channel2 = ort.controlpoints(data_2.x,data_2.y,data_2.z,data_2.w,data_2.array)
channel3 = ort.controlpoints(data_3.x,data_3.y,data_3.z,data_3.w,data_3.array)

## Generation surface $St$

In [ ]:
# Points of the channel surfaces St
evalpts  = NURBS.Bspline(channel1)
evalpts1 = NURBS.Bspline(channel2)
evalpts2 = NURBS.Bspline(channel3)

## Vertical projection of $2D$ grid on $St$

In [5]:
# The risolution for each horizon, top to bottom 
step = 50 ##### THIS NUMBER CAN BE CHANGED !!!!!

In [ ]:
# 2D grid surface 
# N.B. THIS RUNNING TAKES A LOT OF TIME, YOU CAN GO TO GRAB A CUP OF COFFEE
st1 = al.projection(evalpts,step)
st2 = al.projection(evalpts1,step)
st3 = al.projection(evalpts2,step)

In [ ]:
# Make a list of 2d arrays 
# The erosion is introduced to consider the depositional sequence
grid = al.erode([st1,st2,st3])

In [3]:
layer1 = np.ones((393,750))*(8033.02915016)
layer2 = np.ones((393,750))*(8029.054525636)
layer3 = np.ones((393,750))*(8025.04309491)
top    = np.ones((393,750))*(8000)
bottom = np.ones((393,750))*(7950)
quote = 16000 

regular_grid = top, -layer3+quote, -layer2+quote, -layer1+quote, bottom

## Add top and bottom layer

In [ ]:
# Dimension of the array
n,m = st1.shape
# Add two array as bottom and top layer with a specified altitude
top_layer = np.ones((n,m))*(8000)
bottom_layer = np.ones((n,m))*(7950)
# Build the reservoir grid model
# To export in GoCad or ECLIPSE we stack the layers upside down
altitude = 16000 # I add an altitude quote, but that value is not mandatory and it can be changed
reservoir = top_layer, -grid[2]+altitude, -grid[1]+altitude, -grid[0]+altitude, bottom_layer

## Exporting for Eclipse

In [6]:
# Define the origin of 2D grid    
gridplane = al.grid2D( origin=[356607,4795792], step=step, npoints=[393,750] )
# Exporting the block
ecl.write_eclipse( regular_grid, gridplane, "../Results/Eclipse_RegularGrid_5m.GRDECL" )